In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

In [2]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/german.data-numeric

--2021-03-31 12:24:20--  https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/german.data-numeric
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 102000 (100K) [application/x-httpd-php]
Saving to: ‘german.data-numeric’

german.data-numeric 100%[===================>]  99.61K   515KB/s    in 0.2s    

2021-03-31 12:24:21 (515 KB/s) - ‘german.data-numeric’ saved [102000/102000]



In [3]:
arr = np.loadtxt('german.data-numeric')
arr.shape

(1000, 25)

In [4]:
# !wget https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/german.doc
# # with open('german.doc') as f:
# #     lines = f.readlines()
# # lines

In [5]:
X, y = arr[:, :24], arr[:, -1]
y[y == 2] = 0
X.shape, y.shape

((1000, 24), (1000,))

### 80-20 Split

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

D = 24
X_train_0 = X_train[y_train == 0]
X_train_1 = X_train[y_train == 1]
X_test_0 = X_test[y_test == 0]
X_test_1 = X_test[y_test == 1]

M_0 = X_train_0.mean(axis=0)
M_1 = X_train_1.mean(axis=0)

cov_0 = np.zeros((D, D))
for i in range(X_train_0.shape[0]):
    W = (X_train_0[i] - M_0).reshape(-1, 1)
    cov_0 += np.matmul(W, W.T)

cov_1 = np.zeros((D, D))
for i in range(X_train_1.shape[0]):
    W = (X_train_1[i] - M_1).reshape(-1, 1)
    cov_1 += np.matmul(W, W.T)

S_w = cov_0 + cov_1
S_w_inv = np.linalg.inv(S_w)
W = np.matmul(S_w_inv, (M_1 - M_0)).reshape(-1, 1)
W = W/np.linalg.norm(W,2)

def predict(x, W, b):
    return np.matmul(W.T, x) + b


# Line Search
best_b, best_acc = 0, 0
lower = np.matmul(W.T, M_0)[0] - 5
upper = np.matmul(W.T, M_1)[0] + 5
for b in np.linspace(lower, upper, 1000):
    acc = 0
    for (i, X_i) in enumerate(X_train_0):
        pred = predict(X_i.reshape(-1, 1), W, b)
        if pred < 0:
            acc += 1

    for (i, X_i) in enumerate(X_train_1):
        pred = predict(X_i.reshape(-1, 1), W, b)
        if pred >= 0:
            acc += 1

    if acc > best_acc:
        best_acc = acc
        best_b = b
    
    # print(f'Bias {b}, Acc: {acc/X_train.shape[0]}, Best: {best_acc/X_train.shape[0]}')

print('Train acc. =', best_acc/X_train.shape[0])

b = best_b
acc = 0
for (i, X_i) in enumerate(X_test_0):
    pred = predict(X_i.reshape(-1, 1), W, b)
    if pred < 0:
        acc += 1

for (i, X_i) in enumerate(X_test_1):
    pred = predict(X_i.reshape(-1, 1), W, b)
    if pred >= 0:
        acc += 1

print(f'Test acc. = {acc/X_test.shape[0]}')

# f1-score
y_pred = []
for X_i in X_train:
    pred = predict(X_i.reshape(-1, 1), W, b)
    if pred < 0:
        y_pred.append(0)
    else:
        y_pred.append(1)
y_pred = np.array(y_pred)
print('F1-Score (train) = ', f1_score(y_train, y_pred))

y_pred = []
for X_i in X_test:
    pred = predict(X_i.reshape(-1, 1), W, b)
    if pred < 0:
        y_pred.append(0)
    else:
        y_pred.append(1)
y_pred = np.array(y_pred)

print('F1-Score (test) = ', f1_score(y_test, y_pred))
confusion_matrix(y_test, y_pred)

Train acc. = 0.7825
Test acc. = 0.765
F1-Score (train) =  0.8540268456375838
F1-Score (test) =  0.8339222614840989


array([[ 35,  33],
       [ 14, 118]])

### 70-30 Split

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

D = 24
X_train_0 = X_train[y_train == 0]
X_train_1 = X_train[y_train == 1]
X_test_0 = X_test[y_test == 0]
X_test_1 = X_test[y_test == 1]

M_0 = X_train_0.mean(axis=0)
M_1 = X_train_1.mean(axis=0)

cov_0 = np.zeros((D, D))
for i in range(X_train_0.shape[0]):
    W = (X_train_0[i] - M_0).reshape(-1, 1)
    cov_0 += np.matmul(W, W.T)

cov_1 = np.zeros((D, D))
for i in range(X_train_1.shape[0]):
    W = (X_train_1[i] - M_1).reshape(-1, 1)
    cov_1 += np.matmul(W, W.T)

S_w = cov_0 + cov_1
S_w_inv = np.linalg.inv(S_w)
W = np.matmul(S_w_inv, (M_1 - M_0)).reshape(-1, 1)
W = W/np.linalg.norm(W,2)

def predict(x, W, b):
    return np.matmul(W.T, x) + b


# Line Search
best_b, best_acc = 0, 0
lower = np.matmul(W.T, M_0)[0] - 5
upper = np.matmul(W.T, M_1)[0] + 5
for b in np.linspace(lower, upper, 1000):
    acc = 0
    for (i, X_i) in enumerate(X_train_0):
        pred = predict(X_i.reshape(-1, 1), W, b)
        if pred < 0:
            acc += 1

    for (i, X_i) in enumerate(X_train_1):
        pred = predict(X_i.reshape(-1, 1), W, b)
        if pred >= 0:
            acc += 1

    if acc > best_acc:
        best_acc = acc
        best_b = b
    
    # print(f'Bias {b}, Acc: {acc/X_train.shape[0]}, Best: {best_acc/X_train.shape[0]}')

print('Train acc. =', best_acc/X_train.shape[0])

b = best_b
acc = 0
for (i, X_i) in enumerate(X_test_0):
    pred = predict(X_i.reshape(-1, 1), W, b)
    if pred < 0:
        acc += 1

for (i, X_i) in enumerate(X_test_1):
    pred = predict(X_i.reshape(-1, 1), W, b)
    if pred >= 0:
        acc += 1

print(f'Test acc. = {acc/X_test.shape[0]}')

# f1-score
y_pred = []
for X_i in X_train:
    pred = predict(X_i.reshape(-1, 1), W, b)
    if pred < 0:
        y_pred.append(0)
    else:
        y_pred.append(1)
y_pred = np.array(y_pred)
print('F1-Score (train) = ', f1_score(y_train, y_pred))

y_pred = []
for X_i in X_test:
    pred = predict(X_i.reshape(-1, 1), W, b)
    if pred < 0:
        y_pred.append(0)
    else:
        y_pred.append(1)
y_pred = np.array(y_pred)

print('F1-Score (test) = ', f1_score(y_test, y_pred))
confusion_matrix(y_test, y_pred)

Train acc. = 0.7842857142857143
Test acc. = 0.7866666666666666
F1-Score (train) =  0.8549471661863592
F1-Score (test) =  0.8558558558558559


array([[ 46,  41],
       [ 23, 190]])

In [13]:
from sklearn.preprocessing import PolynomialFeatures

pf = PolynomialFeatures(degree=2)
X_poly = pf.fit_transform(X)
X_poly.shape

(1000, 325)

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X_poly, y, test_size=0.3)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

D = X_train.shape[-1]
X_train_0 = X_train[y_train == 0]
X_train_1 = X_train[y_train == 1]
X_test_0 = X_test[y_test == 0]
X_test_1 = X_test[y_test == 1]

M_0 = X_train_0.mean(axis=0)
M_1 = X_train_1.mean(axis=0)

cov_0 = np.zeros((D, D))
for i in range(X_train_0.shape[0]):
    W = (X_train_0[i] - M_0).reshape(-1, 1)
    cov_0 += np.matmul(W, W.T)

cov_1 = np.zeros((D, D))
for i in range(X_train_1.shape[0]):
    W = (X_train_1[i] - M_1).reshape(-1, 1)
    cov_1 += np.matmul(W, W.T)

S_w = cov_0 + cov_1
S_w_inv = np.linalg.pinv(S_w)
W = np.matmul(S_w_inv, (M_1 - M_0)).reshape(-1, 1)
W = W/np.linalg.norm(W,2)

def predict(x, W, b):
    return np.matmul(W.T, x) + b


# Line Search
best_b, best_acc = 0, 0
lower = np.matmul(W.T, M_0)[0] - 5
upper = np.matmul(W.T, M_1)[0] + 5
for b in np.linspace(lower, upper, 1000):
    acc = 0
    for (i, X_i) in enumerate(X_train_0):
        pred = predict(X_i.reshape(-1, 1), W, b)
        if pred < 0:
            acc += 1

    for (i, X_i) in enumerate(X_train_1):
        pred = predict(X_i.reshape(-1, 1), W, b)
        if pred >= 0:
            acc += 1

    if acc > best_acc:
        best_acc = acc
        best_b = b
    
    # print(f'Bias {b}, Acc: {acc/X_train.shape[0]}, Best: {best_acc/X_train.shape[0]}')

print('Train acc. =', best_acc/X_train.shape[0])

b = best_b
acc = 0
for (i, X_i) in enumerate(X_test_0):
    pred = predict(X_i.reshape(-1, 1), W, b)
    if pred < 0:
        acc += 1

for (i, X_i) in enumerate(X_test_1):
    pred = predict(X_i.reshape(-1, 1), W, b)
    if pred >= 0:
        acc += 1

print(f'Test acc. = {acc/X_test.shape[0]}')

# f1-score
y_pred = []
for X_i in X_train:
    pred = predict(X_i.reshape(-1, 1), W, b)
    if pred < 0:
        y_pred.append(0)
    else:
        y_pred.append(1)
y_pred = np.array(y_pred)
print('F1-Score (train) = ', f1_score(y_train, y_pred))

y_pred = []
for X_i in X_test:
    pred = predict(X_i.reshape(-1, 1), W, b)
    if pred < 0:
        y_pred.append(0)
    else:
        y_pred.append(1)
y_pred = np.array(y_pred)

print('F1-Score (test) = ', f1_score(y_test, y_pred))
confusion_matrix(y_test, y_pred)

Train acc. = 0.8942857142857142
Test acc. = 0.6966666666666667
F1-Score (train) =  0.924949290060852
F1-Score (test) =  0.7719298245614036


array([[ 55,  37],
       [ 54, 154]])